In [ ]:
#https://colab.research.google.com/github/ml6team/quick-tips/blob/main/nlp/2021_04_22_shap_for_huggingface_transformers/explainable_transformers_using_shap.ipynb#scrollTo=7XgsySD-u_rN

import shap
import transformers

from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          TextClassificationPipeline)

import os
import pandas as pd
import numpy as np
import helpers

from sklearn.model_selection import train_test_split


/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_min

In [ ]:
df = pd.read_csv('./data/cub.csv', index_col=0)

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1234)

In [ ]:
shap_df = df_test[df_test['species'] == 'yeasts288c']

In [ ]:
import helpers

shap_df = helpers.add_codons_to_df(shap_df, 'Sequence')

In [ ]:
lengths = [len(seq) for seq in shap_df['codons_cleaned']]
shap_df['lengths'] = lengths

In [ ]:
shap_df = shap_df[shap_df['lengths'] < 1064]

In [ ]:
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots()
# N, bins, patches = ax.hist(np.log(df['median_exp']), edgecolor='white', linewidth=1, bins=30)

# for i in range(0,12):
#     patches[i].set_facecolor('#068cf9')
# # for i in range(7,14):    
# #     patches[i].set_facecolor('#a41daa')
# for i in range(12, len(patches)):
#     patches[i].set_facecolor('#ff0051')

# plt.title('Distribution of Median Expression')
# plt.xlabel('Log of Median Expression')
# plt.ylabel('Bin Count')
# plt.show()

In [ ]:
pos = max(df['median_exp'])
neutral = np.median(df['median_exp'])
neg = min(df['median_exp'])

In [ ]:
pos

In [ ]:
neutral

In [ ]:
neg

In [ ]:
pos_seq = ''.join(df.loc[df['median_exp'] == pos, 'codons_cleaned'].tolist())
pos_seq

In [ ]:
neutral_seq = ''.join(df.loc[df['median_exp'] == 14.0, 'codons_cleaned'].tolist())
neutral_seq

In [ ]:
neg_seq = ''.join(df.loc[df['median_exp'] == neg, 'codons_cleaned'].tolist())
neg_seq

In [ ]:
tokenizer_name = "./tokenizers/codonBERT"
model_name = "./models/codonBERT-binary-large_1/checkpoint-127330"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, token_type_ids=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name).cpu()

pipe_binary = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

In [ ]:
tokenizer_name = "./tokenizers/codonBERT"
model_name = "./models/codonBERT-multi-large_1/checkpoint-127330"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, token_type_ids=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name).cpu()

pipe_multi = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

In [ ]:
def score_and_visualize(texts, pipe):
    #prediction = pipe(texts)
    #print(prediction[0])

    explainer = shap.Explainer(pipe)
    shap_values = explainer(texts)

    #shap.plots.text(shap_values)
    shap.plots.beeswarm(shap_values)

In [ ]:
score_and_visualize(shap_df['codons_cleaned'], pipe_binary)

In [ ]:
score_and_visualize(shap_df['codons_cleaned'], pipe_multi)

In [ ]:
def score_and_visualize(texts, pipe):
    prediction = pipe([texts])
    print(prediction[0])

    explainer = shap.Explainer(pipe)
    shap_values = explainer([texts])

    #shap.plots.text(shap_values)
    #shap.plots.beeswarm(shap_values)
    shap.plots.bar(shap_values[0])

In [ ]:
#positive
score_and_visualize('ATG ACA CGC GTT CAA TTT AAA CAC CAC CAT CAT CAC CAT CAT CCT GAC TAG')

In [ ]:
#negative
score_and_visualize('ATG CCA GTC AAC AGC GTC CTT TGC CAT TTT TCT TCC GAC TTT TCA TTG GGC CTC ATA TTG CAA GAT ATC TAA')

In [ ]:
#positive
score_and_visualize('ATG ACA CGC GTT CAA TTT AAA CAC CAC CAT CAT CAC CAT CAT CCT GAC TAG')

In [ ]:
#neutral low
score_and_visualize('ATG AAC GCG GCG ATA TTG AAA ATT CGC TTC AGT TTC CAA GGA TTT CTT GAA AGA GAT TAGATG TGC CGT CGA GAG AGG TGG TTG GTA CGG TAG')

In [ ]:
#neutral med
score_and_visualize('ATG GTT GAT CCA TAC TGG ATG GCA AAC TTC AAC TGT TAG')

In [ ]:
#neutral high
score_and_visualize('ATG TTT GAA CTG TTT GTT ACA AAT GAC ATC TCT CAC TCC TGA')

In [ ]:
#negative
score_and_visualize('ATG CCA GTC AAC AGC GTC CTT TGC CAT TTT TCT TCC GAC TTT TCA TTG GGC CTC ATA TTG CAA GAT ATC TAA')